In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from collections import Counter
import random
from utils import create_dataset
from utils import load_model, save_checkpoint
from utils import AveragePrecision
from torch.utils.data import DataLoader, TensorDataset

## PreProcess

In [2]:
rating_data = pd.read_csv("data/rating_train.csv")
rating_data = rating_data.drop(['date'], axis=)

/home/victai/.local/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [3]:
data_cnt = {}
for i in user_ids:
    data_cnt[i] = len(rating_data[rating_data['userid'] == i])

#plt.hist(list(data_cnt.values()), bins=20)
#list(data_cnt.values())

In [4]:
user_food_score = [{} for i in range(max(user_ids)+1)]
for u, f in zip(rating_data['userid'], rating_data['foodid']):
    if user_food_score[u].get(f) != None:
        user_food_score[u][f] += 1
    else:
        user_food_score[u][f] = 1
    
for i in user_ids:
    all_val = list(user_food_score[i].values())
    for k, v in user_food_score[i].items():
        user_food_score[i][k] = (v - min(all_val)) / (max(all_val) - min(all_val))

In [5]:
valid_percentage = 0.5
user_id_train, user_id_valid, food_id_train, food_id_valid, Y_train, Y_valid = create_dataset(user_food_score, rating_data, valid_percentage)

order = np.arange(len(user_id_train))
random.seed(0)
random.shuffle(order)

In [6]:
not_eaten = {}
for u in user_ids:
    t_start_idx = np.where(user_id_train == u)[0][0]
    t_end_idx = np.where(user_id_train == u)[0][-1]
    v_start_idx = np.where(user_id_valid == u)[0][0]
    v_end_idx = np.where(user_id_valid == u)[0][-1]
    #tmp = []
    #for i in range(v_start_idx, v_end_idx+1):
        #if (food_id_valid[i] not in food_id_train[t_start_idx: t_end_idx+1]) and (food_id_valid[i] not in tmp):
        #    tmp.append(food_id_valid[i])
    #print(u)
    #print(t_end_idx)
    not_eaten[u] = np.setdiff1d(food_id_valid[v_start_idx: v_end_idx+1], food_id_train[t_start_idx: t_end_idx+1])

## Model

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_foods, n_dim):
        super(MatrixFactorization, self).__init__()
        self.user_embed_layer = nn.Embedding(n_users, n_dim).cuda()
        self.food_embed_layer = nn.Embedding(n_foods, n_dim).cuda()
        self.user_bias_embed_layer = nn.Embedding(n_users, 1).cuda()
        self.food_bias_embed_layer = nn.Embedding(n_foods, 1).cuda()
        
    def forward(self, user_train, food_train):
        user_vec = self.user_embed_layer(user_train.cuda())
        user_vec = nn.Dropout(0.3)(user_vec)
        user_vec = nn.Sigmoid()(user_vec)
        food_vec = self.food_embed_layer(food_train.cuda())
        food_vec = nn.Dropout(0.3)(food_vec)
        food_vec = nn.Sigmoid()(food_vec)
        user_bias = self.user_bias_embed_layer(user_train.cuda()).view(-1)
        food_bias = self.food_bias_embed_layer(food_train.cuda()).view(-1)
        
        lamb = 0.01
        reg1 = lamb * torch.norm(torch.cat([x.view(-1) for x in self.parameters()]), 1)
        reg2 = lamb * torch.norm(torch.cat([x.view(-1) for x in self.parameters()]), 2)
        
        return ((user_vec * food_vec).sum(1) + user_bias + food_bias).cuda(), reg1+reg2

In [18]:
n_dim = 10
model = MatrixFactorization(max(user_ids)+1, food_data.shape[0], n_dim).cuda()
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.002)

## Training

In [ ]:
epochs = 10000
batch_size = 512
save_every = 10
print_every = 10
infer_every = 20
best = 1e10
best_epoch = 1
best_map = 0
for e in range(1, epochs+1):
    if e - best_epoch > 10: break
    t_loss = 0
    for b in range(len(user_id_train) // batch_size):
        optimizer.zero_grad()
        u_train = Variable(torch.from_numpy(user_id_train[b*batch_size: (b+1)*batch_size]).type(torch.LongTensor))
        f_train = Variable(torch.from_numpy(food_id_train[b*batch_size: (b+1)*batch_size]).type(torch.LongTensor))
        
        pred, regu_loss = model(u_train, f_train).cuda()
        loss = loss_func(pred, Variable(torch.from_numpy(Y_train[b*batch_size: (b+1)*batch_size]).float()).cuda())+regu_loss
        t_loss += loss
        loss.backward()
        optimizer.step()
    t_loss /= (len(user_id_train) // batch_size)
    t_loss = t_loss.cpu().data.numpy()

    # validation
    v_pred = model(Variable(torch.LongTensor(user_id_valid.tolist())), Variable(torch.LongTensor(food_id_valid.tolist()))).cuda()
    v_loss = loss_func(v_pred, Variable(torch.from_numpy(Y_valid).float()).cuda()).cpu().data.numpy()
    
    if e % infer_every == 0:
        all_ap, _ = infer()
        max_ap = np.array(all_ap).max()
        mean_ap = np.array(all_ap).mean()
        print("max: {}, mean: {}".format(max_ap, mean_ap))
        if mean_ap > best_map:
            best_map = mean_ap
    
    if e % save_every == 0:
        save_checkpoint({
            'epoch': e,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'val_best': best
            },
            False,
            'model.h5'
        )
    if v_loss < best:
        save_checkpoint({
            'epoch': e,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'val_best': v_loss
            },
            True,
            'model.h5'
        )
        best = v_loss
        best_epoch = e
    
    if e % print_every == 0:
        print("Epoch {}, train_loss = {}, valid_loss = {}, best = {}, best_map = {}".format(e, t_loss, v_loss, best, best_map))
print("best_epoch: {}, best_valid_loss: {}, best_map: {}".format(best_epoch, best, best_map))

Epoch 10, train_loss = 0.578853964805603, valid_loss = 0.5194810628890991, best = 0.5194810628890991, best_map = 0
max: 0.0, mean: 0.0
Epoch 20, train_loss = 0.17961128056049347, valid_loss = 0.1657111942768097, best = 0.1657111942768097, best_map = 0
Epoch 30, train_loss = 0.08602795004844666, valid_loss = 0.08283458650112152, best = 0.08283458650112152, best_map = 0
max: 0.0, mean: 0.0
Epoch 40, train_loss = 0.06296724081039429, valid_loss = 0.06226726621389389, best = 0.06226726621389389, best_map = 0
Epoch 50, train_loss = 0.05498889461159706, valid_loss = 0.05483543500304222, best = 0.05483543500304222, best_map = 0
max: 0.0, mean: 0.0
Epoch 60, train_loss = 0.0508057177066803, valid_loss = 0.0508696585893631, best = 0.0508696585893631, best_map = 0
Epoch 70, train_loss = 0.048136156052351, valid_loss = 0.04832153022289276, best = 0.04832153022289276, best_map = 0
max: 0.0, mean: 0.0
Epoch 80, train_loss = 0.04641120508313179, valid_loss = 0.04657445102930069, best = 0.04657445102

In [9]:
model,_,_,_ = load_model('model.h5.best', model)

=> loading checkpoint model.h5.best
=> loaded checkpoint model.h5.best (epoch: 252)


## Inference

In [9]:
def infer(): 
    F = np.arange(len(food_data), dtype=int)
    F = [i for i in range(len(food_data))]
    all_ans = []
    all_score = []
    for u in user_ids:
        ans = []
        U = [u for i in range(len(food_data))]
        res = model(Variable(torch.LongTensor(U)), Variable(torch.LongTensor(F))).cpu().data.numpy()
        res = res.flatten()
        D = {k: v for (k, v) in zip(F, res)}
        sorted_D = sorted(D.items(), key=lambda d: d[1], reverse=True)
        for x in sorted_D:
            if len(ans) >= 20: break
            if x[0] in user_food_score[u].keys():
                continue
            ans.append(x[0])
        all_score.append(AveragePrecision(not_eaten[u], ans, len(ans)))
        all_ans.append(ans) 
    return all_score, all_ans

In [16]:
not_eaten[6]

array([122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175])

## Generate Output

In [63]:
with open("pred.csv", 'w') as f:
    f.write("userid,foodid\n")
    for i in range(len(user_ids)):
        f.write("{},".format(user_ids[i]))
        for j in all_ans[i]:
            f.write("{} ".format(j))
        f.write('\n')